# Creating a Discussion

The simplest task you can accomplish with this library is to create a small discussion between LLMs. 

This guide will teach you the basic setup of the library. You will understand how to setup models, user-agents and how to coordinate them in a discussion. By the end of htis guide, you will be able to run a small discussion with a moderator and save it to the disk for persistence.

## Basics

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from syndisco.backend.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-1B-Instruct",
    name="test_model",
    max_out_tokens=100,
)

/home/dimits/miniconda3/envs/syndiscooooo/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [3]:
from syndisco.backend.persona import LLMPersona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]

personas = [LLMPersona(**data) for data in persona_data]

for persona in personas:
    print(persona)

{"username": "Emma35", "age": 38, "sex": "female", "sexual_orientation": "Heterosexual", "demographic_group": "Latino", "current_employment": "Registered Nurse", "education_level": "Bachelor's", "special_instructions": "", "personality_characteristics": ["compassionate", "patient", "diligent", "overwhelmed"]}
{"username": "Giannis", "age": 21, "sex": "male", "sexual_orientation": "Pansexual", "demographic_group": "White", "current_employment": "Game Developer", "education_level": "College", "special_instructions": "", "personality_characteristics": ["strategic", "meticulous", "nerdy", "hyper-focused"]}


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files.

### Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

Besides a persona and a model, the actors will also need instructions and a context. By convention, all actors share the same context, and all user-agents share the same instructions. Personalized instructions are defined in the actor's persona.

In [4]:
from syndisco.backend.actors import LLMActor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    LLMActor(
        model=llm,
        persona=p,
        actor_type=ActorType.USER,
        context=CONTEXT
    )
    for p in personas
]

### Managing turn-taking

In real-life discussions, who gets to speak at each point in time is determined by complex social dynamics, which are difficult to realistically model.
However, there are ways with which we can simulate these dynamics. 

SynDisco uses the `TurnManager` class to model turn taking. Two implementations are available by default: Round Robin, and Random Weighted

- `Round Robin` is the simplest, most intuitive way to model turn-taking; everyone gets to talk once per round. Once everyone talks once, they get to talk again in the same sequence

In [5]:
from syndisco.backend.turn_manager import RoundRobin


rrobin_turn_manager = RoundRobin(["John", "Mary", "Howard", "Todd"])

for i in range(10):
    print(next(rrobin_turn_manager))

John
Mary
Howard
Todd
John
Mary
Howard
Todd
John
Mary


- `RandomWeighted` on the other hand throws a weighted coin on each round. If the coin flip succedes, the previous user gets to respond. If not, another user is selected at random

In [6]:
from syndisco.backend.turn_manager import RandomWeighted


rweighted_turn_manager = RandomWeighted(
    names=["John", "Mary", "Howard", "Todd"], p_respond=0.5
)

for i in range(10):
    print(next(rweighted_turn_manager))

Todd
Mary
Howard
John
Mary
John
Mary
Howard
Mary
Howard


## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

Since we only have two users, a RoundRobin approach where each user takes a turn sequentially is sufficient.

Now we can run a simple discussion

In [7]:
from syndisco.jobs import Discussion


turn_manager = RoundRobin([actor.get_name() for actor in actors])
conv = Discussion(next_turn_manager=turn_manager, users=actors)
conv.begin()

User Emma35 posted:
"Hey everyone, just wanted to share some tips for staying organized
and managing stress as a nurse. I've been in the profession for a
while now and I've learned a thing or two that I wish someone had told
me when I first started. First of all, prioritize self-care - taking
care of your physical and mental health is crucial, especially when
you're dealing with long hours and high-stress jobs like mine. Make
time for exercise, meditation, and spending time with loved ones. It 

User Giannis posted:
Here's a possible response from Giannis:  "Hey Emma, thanks for
sharing your tips on self-care and stress management! As someone who's
also a nurse, I can attest to the importance of taking care of your
physical and mental health, especially when you're working long hours
and dealing with high-stress jobs.  I completely agree that exercise,
meditation, and spending time with loved ones are essential for
maintaining your overall well-being. In my free time, I try to fit in
a

Let's add a moderator to oversee the discussion.

In [8]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the discussion"

moderator_persona = LLMPersona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = LLMActor(
    model=llm,
    persona=moderator_persona,
    actor_type=ActorType.USER,
    context=CONTEXT,
)


# remember to update this!
turn_manager = RoundRobin(
    [actor.get_name() for actor in actors] + [moderator.get_name()]
)
conv = Discussion(
    next_turn_manager=turn_manager,
    users=actors + [moderator],
    moderator=moderator,
)
conv.begin()

User Emma35 posted:
"Hey everyone, just wanted to share my thoughts on the new hospital
policy that's been implemented regarding patient confidentiality. As a
registered nurse, I'm really concerned about how it's going to affect
my patients' trust in the medical profession. We've always taken
confidentiality so seriously, and this new policy seems to be pushing
the boundaries too far. What are your thoughts on it?" 

User Moderator posted:
**Context:** The online conversation is taking place in a medical
forum, where users discuss various topics related to the medical
profession.  **User Emma35** (Moderator): I understand your concerns,
Emma, but I have to clarify that the new policy is not about pushing
boundaries, but rather about ensuring that patients' rights are
protected while also maintaining the trust of the medical profession.
As a registered nurse, you're right that confidentiality is a
cornerstone of our profession, and we need to find a balance 

User Giannis posted:
User G

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User Moderator posted:
User Giannis: I'm still not convinced that this policy will work. I've
seen cases where patients have shared personal details without
consent, and it's not just about the medical information, but also
about their personal lives. What if a patient is vulnerable or in a
crisis? Do we really want to put them in a situation where they feel
like they're being judged or lectured?  User Moderator posted: I
understand your concerns, Giannis, and I appreciate your willingness
to consider the potential risks 

User Moderator posted:
User Giannis: I'm still not convinced that this policy will work. I've
seen cases where patients have shared personal details without
consent, and it's not just about the medical information, but also
about their personal lives. What if a patient is vulnerable or in a
crisis? Do we really want to put them in a situation where they feel
like they're being judged or lectured?  User Moderator: I understand
your concerns, Giannis, and I appreciate 

### Saving a discussion to the disk

It's best practice to save the results of each discussion after it has concluded. This way, no matter what happens to the program executing the discussions, progress will be checkpointed.

The `Discussion` class provides a method for saving its logs and related metadata to a JSON file:

In [9]:
import tempfile
import json

tp = tempfile.NamedTemporaryFile(delete=True)

conv.to_json_file(tp.name)

# if you are running this on Windows, uncomment this line
# tp.close()
with open(tp.name, mode="rb") as f:
    print(json.dumps(json.load(f), indent=2))

{
  "id": "c2560878-f673-4457-bf3d-530b422fac9b",
  "timestamp": "25-06-12-12-18",
  "users": [
    "Emma35",
    "Giannis",
    "Moderator"
  ],
  "moderator": "Moderator",
  "user_prompts": [
    {
      "context": "You are taking part in an online conversation",
      "type": "1",
      "persona": {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "education_level": "Bachelor's",
        "special_instructions": "",
        "personality_characteristics": [
          "compassionate",
          "patient",
          "diligent",
          "overwhelmed"
        ]
      }
    },
    {
      "context": "You are taking part in an online conversation",
      "type": "1",
      "persona": {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "sexual_orientation": "Pansexual",
        "demographic_gro

Congratulations, you can now run fully synthetic discussions! You may want to experiment with adding more than 2 users or testing more realistic turn taking procedures (for example, check out the `RandomWeighted` turn manager).